# classification_dataset

### Imports

In [6]:
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import cv2 as cv
import json

sys.path.insert(0, "../../packages/python")
from models import cell_segmentation as segmentators

2025-04-17 19:19:44.048886: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-17 19:19:44.054900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744928384.061810   30624 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744928384.063883   30624 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 19:19:44.071551: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Definitions

In [34]:
CSV_PATH = '../../output/cropped_cells_onion_v3/data/valid/'
CROPS_PATH = '../../output/cropped_cells_onion_v3/media/valid/'
IMAGES_PATH = '../../media/Onion-Cell-Merged-v6.v1i.coco/valid/'
JSON_PATH = '../../temp/datasets_area_data.json'
OUTPUT_PATH = '../../output/cropped_cells_onion_v4/media/valid/'
IMG_SIDE = 200


### List of elements to use

In [35]:
csvs = sorted(os.listdir(CSV_PATH)) #Paths to the csv of SAM detections of each image
crops = sorted(os.listdir(CROPS_PATH)) #Paths to the crops made from SAM detection of the full_images
images = sorted(os.listdir(IMAGES_PATH)) #full_images from where the crops are made
with open(JSON_PATH, 'r') as f: #json with the information of the filename of the images
    area_data = json.load(f)

### Dataset generation

In [36]:
resize_factor = IMG_SIDE/area_data['INA']['lado_cuadrado']

for image in tqdm(images):
 
    try:
        image_name, image_type =  image.split('.')
    except Exception as e: 

        print(image, e)
        continue
    image_group = image_name[0] if image_name[0].isalpha() else "INA" 
    image_side = area_data[image_group]['lado_cuadrado']
    image_resize_factor = int(resize_factor * image_side)

    img = cv.imread(os.path.join(IMAGES_PATH, image))

    df = pd.read_csv(os.path.join(CSV_PATH, f"{image_name}.csv"))
    df_bbox = df[df['image'] == image][['x', 'y', 'w', 'h', 'cell_id']]    

    for _, row in df_bbox.iterrows():
        cell_id = row['cell_id']    
        x, y, w, h = row['x'], row['y'], row['w'], row['h']
        x, y, w, h = segmentators.CellMaskGenerator.adjust_bbox(segmentators.CellMaskGenerator, x, y, w, h, image_resize_factor*image_resize_factor, img.shape[1], img.shape[0])

        crop = cv.resize(img[y:y+h, x:x+w], (IMG_SIDE, IMG_SIDE))
        output_path = os.path.join(OUTPUT_PATH, f"{image_name}_{cell_id}.{image_type}")
        cv.imwrite(output_path, crop)





                

100%|██████████| 129/129 [00:08<00:00, 15.55it/s]
